## eBay Baseball Card Data Scraper

In [14]:
import requests
import numpy as np
import pandas as pd
from scipy import stats
from bs4 import BeautifulSoup as bs

In [7]:
#link = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1312&_nkw=lewis+hamilton+chrome+auto&_sacat=0"

# link = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=2019+panini+prizm+zion+williamson+248+psa+10&_pgn=3'
# mode = 'scrape'

In [8]:
# global variables 

keywords = '2019+panini+prizm+zion+williamson+248+psa+10'
base_link = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw='
end_link = '&_pgn='
link = base_link + keywords + end_link

names = []
prices = []
shippings = []
links = []
listing_types = []
grades = []



In [ ]:
def get_grade(title):

    index = ''
    string = ''

    grading_companies = ['PSA', 'psa', 'BGS', 'bgs', 'HGA', 'hga', 'SGC', 'sgc']
    for company in grading_companies:
        index = title.find(company)
        if (index != -1):
            return title[index:index + 7]
    
    return None

def get_sale_type(item):
    

In [38]:
def get_data(link):
    page = requests.get(link)
    soup = bs(page.text, 'html.parser')
    return soup

def parse(soup):
    results = soup.find_all('div', {'class': 's-item__info clearfix'})
    products_list = []

    for item in results:
        products = {
            'title': getattr(item.find('h3', {'class': 's-item__title'}), 'text', None),
            'grade': get_grade(getattr(item.find('h3', {'class': 's-item__title'}), 'text', None)),
            'soldprice': float(item.find('span', {'class': 's-item__price'}).text.replace("$", '').replace(',', '').strip()),
            'solddate': getattr(item.find('div', {'class': 's-item__title--tagblock'}), "POSITIVE", None),
            'saletype': get_sale_type(item)
            #'solddate': item.find('div', {'class': 's-item__title--tagblock'}).find('span', {'class': 'POSITIVE'}).text, ---- needs to be fixed
            'bids': getattr(item.find('span', {'class': 's-item__bids s-item__bidCount'}), 'text', None),
            'link': item.find('a', {'class': 's-item__link'})['href'],
        }
        products_list.append(products)
    return products_list

def output(products):
    df = pd.DataFrame(products)
    df.to_csv('output.csv', index=False)
    return df

soup = get_data('https://www.ebay.com/sch/i.html?_nkw=2019+panini+prizm+zion+williamson+248+psa+9&_sop=13&rt=nc&LH_Sold=1&LH_Complete=1')
products = parse(soup)
frame = output(products)

In [42]:
frame

0          NA
1       PSA 9
2     PSA 9 🔥
3     PSA 9 P
4      PSA 9 
       ...   
56    PSA 9 M
57    psa 9 R
58    PSA 9 🔥
59      PSA 9
60      PSA 9
Name: grade, Length: 61, dtype: object

In [9]:
## functions

def scrape_listings(link):

    full_listings = []
    
    for i in range(1, 7): # needs to be edited statically, will parse from page eventually....

        page = requests.get(link + str(i))
        content = bs(page.content)
        soup = bs(page.content, 'html.parser')
        listings = soup.find_all('li', attrs={'class': 's-item'})
        full_listings.extend(listings)

    return full_listings

def find_name(listing):
    for name in listing.find_all('h3', attrs={'class': "s-item__title"}):
        if (str(name.find(text=True, recursive=False))!="None"):
                return str(name.find(text=True, recursive=False))
        else:
            return 'NA'

def find_grade(title):

    index = ''
    string = ''

    grading_companies = ['PSA', 'psa', 'BGS', 'bgs', 'HGA', 'hga', 'SGC', 'sgc']
    for company in grading_companies:
        index = title.find(company)
        if (index != -1):
            return title[index:index + 7]
    
    return 'NA'  

def find_price(listing):
    price = listing.find('span', attrs={'class':"s-item__price"}).get_text(strip=True)
    price = int(price.replace(',', '')[1:-3])
    return price

def find_ship(listing):
    cost = listing.find('span', attrs={'class':"s-item__shipping", 'class':"s-item__logisticsCost"})
    if cost is not None:
        return str(cost.find(text=True, recursive=False))
    else:
        return 'NA'

def find_type(listing):
    obo = listing.find('span', attrs={'class': 's-item__purchase-options-with-icon'})
    if (obo is not None):
        return 'BIN/OBO'
    else:
        return 'Auction'

def get_info(listing):

    prod_link = listing.select_one('a')['href']
    prod_name = find_name(listing)
    prod_grade = find_grade(prod_name)
    prod_price = find_price(listing)
    prod_ship_cost = find_ship(listing)
    prod_list_type = find_type(listing)

    def append_all():
        links.append(prod_link)
        names.append(prod_name)
        grades.append(prod_grade)
        prices.append(prod_price)
        shippings.append(prod_ship_cost)
        listing_types.append(prod_list_type)
    
    append_all()

In [10]:
listings = scrape_listings(link)

for listing in listings:
    get_info(listing)
    

In [11]:
zion = pd.DataFrame({
    "Titie": names,
    "Grade": grades,
    "Price": prices,
    "Listing Type": listing_types,
    "Shipping Cost": shippings,
    "Link": links})

In [12]:
zion

,Titie,Grade,Price,Listing Type,Shipping Cost,Link
0,Shop on eBay,NA,20,Auction,NA,https://ebay.com/itm/123456?hash=item28caef0a3...
1,ZION WILLIAMSON PRIZM CARD GRADED SGC 10 GEM M...,SGC 10,295,BIN/OBO,Free shipping,https://www.ebay.com/itm/393778243578?epid=705...
2,2019 PANINI PRIZM ZION WILLIAMSON #248 PSA 10!,PSA 10!,1,Auction,+$4.50 shipping,https://www.ebay.com/itm/394193049928?hash=ite...
3,2019-20 Panini Prizm #248 Zion Williamson (RC)...,PSA 10🔥,6,Auction,Free shipping,https://www.ebay.com/itm/284925822354?epid=240...
4,2019-20 Panini Prizm #248 Zion Williamson (RC)...,PSA 10!,51,Auction,+$4.50 shipping,https://www.ebay.com/itm/225099937383?epid=240...
...,...,...,...,...,...,...
274,🔥2019-20 Panini Prizm Zion Williamson #248 PSA...,PSA 10,2000,BIN/OBO,+$5.00 shipping,https://www.ebay.com/itm/143960808582?epid=240...
275,2019 PANINI PRIZM #248 ZION WILLIAMSON PSA 10,PSA 10,849,BIN/OBO,+$4.70 shipping,https://www.ebay.com/itm/293970036209?epid=240...
276,ZION WILLIAMSON 2019 20 PANINI PRIZM #248 ROOK...,PSA 10,399,BIN/OBO,+$4.65 shipping,https://www.ebay.com/itm/394008891806?hash=ite...
277,PSA 10 Gem Mint Zion Williamson RC 2019-20 Pan...,PSA 10,349,BIN/OBO,+$5.50 shipping,https://www.ebay.com/itm/324596456986?epid=240...
